<a href="https://colab.research.google.com/github/hoonio/jupyter/blob/master/Assignment%2B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd
olympics_data_url = 'https://raw.githubusercontent.com/hoonio/jupyter/master/olympics.csv'
census_data_url = 'https://github.com/hoonio/jupyter/raw/master/census.csv'

df = pd.read_csv(olympics_data_url, index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.nlargest(10, 'Gold')

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA
Soviet Union,9,395,319,296,1010,9,78,57,59,194,18,473,376,355,1204,URS
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
China,9,201,146,126,473,10,12,22,19,53,19,213,168,145,526,CHN
Italy,26,198,166,185,549,22,37,34,43,114,48,235,200,228,663,ITA
Germany,15,174,182,217,573,11,78,78,53,209,26,252,260,270,782,GER
Hungary,25,167,144,165,476,22,0,2,4,6,47,167,146,169,482,HUN
East Germany,5,153,129,127,409,6,39,36,35,110,11,192,165,162,519,GDR
Sweden,26,143,164,176,483,22,50,40,54,144,48,193,204,230,627,SWE


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [0]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [6]:
def answer_one():
#     df['country'] = df.index
#     df.set_index("Gold")
#     df.reset_index()
#     return df
    return df.nlargest(5, 'Gold').first_valid_index()

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
import numpy as np

def answer_two():
    summer = df.filter(items=['Gold', 'Gold.1'])
    summer['Diff'] = np.absolute(summer['Gold']-summer['Gold.1'])
#     winter = df.filter(items=['Gold.1'])
    return summer.nlargest(10, 'Diff').first_valid_index()

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [4]:
def answer_three():
    golds = df[(df['Gold']>0) & df['Gold.1']>0]
    golds = golds.filter(items=['Gold', 'Gold.1']).rename(columns={'Gold': 'a', 'Gold.1': 'b'})
    golds['Diff'] = (golds.a-golds.b)
    golds['Sum'] = (golds.a+golds.b)
    golds['Ratio'] = (golds['Diff']/golds['Sum'])
    return golds.nlargest(10, 'Ratio').first_valid_index()

answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [3]:
def answer_four():
#     return df[(df['Gold']>0) | df['Gold.1']>0]
    df['Gold.2'] = (df['Gold'] * 3)
    df['Silver.2'] = (df['Silver'] * 2)
    df['Points'] = (df['Gold'] * 3) + (df['Silver'] * 2) + df['Bronze']
    return df.filter(items=['Points'])

answer_four()

,Points
Afghanistan,2
Algeria,27
Argentina,130
Armenia,16
Australasia,22
Australia,898
Austria,155
Azerbaijan,43
Bahamas,24
Bahrain,1


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [2]:
census_df = pd.read_csv(census_data_url)
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [7]:
def answer_five():
#     census_df
#     return census_df["STNAME"].unique()
#     twotier = census_df.set_index(["STNAME", "CTYNAME"])
    return census_df.groupby("STNAME").nunique().nlargest(10, 'CTYNAME').first_valid_index()
#     states = census_df['STNAME'].unique()
#     states["County"] = census_df["CTYNAME"].nunique()
#     return states
#     return census_df["STNAME"].nunique()

answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [8]:
def answer_six():
#     return census_df.groupby("STNAME").sum().nlargest(10, 'CENSUS2010POP')
#      return census_df.groupby("STNAME")['CENSUS2010POP'].nlargest(3)
    states = census_df.groupby('STNAME')['CENSUS2010POP'].apply(lambda x: x.nlargest(3).sum()).nlargest(3).index.values.tolist()
    return states

answer_six()

['California', 'Texas', 'New York']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [9]:
def answer_seven():
    return census_df[census_df['SUMLEV']==50].set_index('CTYNAME').loc[:,'POPESTIMATE2010':'POPESTIMATE2015'].apply(lambda g: g.max() - g.min(), axis=1).nlargest(5).first_valid_index()
#     return census_df[census_df['SUMLEV']==50].set_index('CTYNAME').loc[:,'POPESTIMATE2010':'POPESTIMATE2015'].max(axis=1)

answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [10]:
def answer_eight():
    return census_df[census_df['REGION']<=2].groupby('STNAME')['CTYNAME']

answer_eight()